In [1]:
#!/usr/bin/python3
import random

import numpy as np
import pandas as pd

from numpy.random import beta
from typing import List

from pandas.core.frame import DataFrame #class
from pandas.core.series import Series #class

In [3]:
modelDF = pd.DataFrame({"votes":[0.5, 0.4, 0.1]}, index=["method1","method2","method3"])
modelDF

,votes
method1,0.5
method2,0.4
method3,0.1


In [4]:
userID = 0

In [8]:
numberOfItems = 5

In [10]:
methodsResultDict = {
    "method1": {1:0.5, 2:0.5, 3:0.4, 4:0.2},
    "method2": {5:0.9, 1:0.5, 6:0.1},
    "method3": {3:0.7, 4:0.3, 6:0.9}
}

In [65]:
def run( methodsResultDict:dict, modelDF:DataFrame, userID:int, numberOfItems:int = 20):

      # testing types of parameters
      if type(methodsResultDict) is not dict:
          raise ValueError("Type of methodsResultDict isn't dict.")
      if type(modelDF) is not DataFrame:
          raise ValueError("Type of methodsParamsDF isn't DataFrame.")
      if list(modelDF.columns) != ['votes']:
          raise ValueError("Argument methodsParamsDF doen't contain rights columns.")
      if type(numberOfItems) is not int:
          raise ValueError("Type of numberOfItems isn't int.")

      if sorted([mI for mI in modelDF.index]) != sorted([mI for mI in methodsResultDict.keys()]):
        raise ValueError("Arguments methodsResultDict and methodsParamsDF have to define the same methods.")
      for mI in methodsResultDict.keys():
          if modelDF.loc[mI] is None:
              raise ValueError("Argument modelDF contains in ome method an empty list of items.")
      if numberOfItems < 0:
        raise ValueError("Argument numberOfItems must be positive value.")

      candidatesOfMethods = [np.array(list(cI.keys())) for cI in methodsResultDict.values()]
      uniqueCandidatesI:List[int] = list(set(np.concatenate(candidatesOfMethods)))
      print("UniqueCandidatesI: ", uniqueCandidatesI)    

      # sum of preference the elected candidates have for each party
      electedOfPartyDictI:dict[str,float] = {mI:0.0 for mI in modelDF.index}
      print("ElectedForPartyI: ", electedOfPartyDictI)

      
      # votes number of parties
      votesOfPartiesDictI:dict[str,float] = {mI:modelDF.votes.loc[mI] for mI in modelDF.index}
      totalVotes = modelDF["votes"].sum()
      print("VotesOfPartiesDictI: ", votesOfPartiesDictI, totalVotes)

      recommendedItemIDs:List[int] = []
      
      totalSelectedCandidatesVotes:float = 0.0
            
    
      iIndex:int
      for iIndex in range(0, numberOfItems):
        #print("iIndex: ", iIndex)
        
        if len(uniqueCandidatesI) == 0:
            return recommendedItemIDs[:numberOfItems]

        # coumputing of votes of remaining candidates
        actVotesOfCandidatesDictI:dict[int,int] = {} #calculates the proportonal improvement of the output if this candidate is included
        candidateIDJ:int
        for candidateIDJ in uniqueCandidatesI:
           votesOfCandidateJ:float = 0.0
          
           candidateVotesPerParty:dict[str,float] = {mI:methodsResultDict[mI].get(candidateIDJ, 0) for mI in modelDF.index}            
           candidateTotalVotes:float = np.sum(list(candidateVotesPerParty.values()))
           totalVotesPlusProspected:float = totalSelectedCandidatesVotes + candidateTotalVotes                      
            
           for parityIDK in modelDF.index:   
              #get the fraction of under-representation for the party
              #check how much proportional representation the candidate adds
              #sum over all parties & select the highest sum
              votes_fraction_per_party = votesOfPartiesDictI[parityIDK]/totalVotes
              notRepresentedVotesPerParty = max(0,(votes_fraction_per_party * totalVotesPlusProspected) - electedOfPartyDictI[parityIDK]) #max(w_i*(A+C) - a_i,0)
              #print("Unrepresented,",parityIDK,candidateIDJ,notRepresentedVotesPerParty)
              votesOfCandidateJ += min(notRepresentedVotesPerParty, candidateVotesPerParty[parityIDK]  ) # only account the amount of votes that does not exceed proportional representation            

           actVotesOfCandidatesDictI[candidateIDJ] = votesOfCandidateJ
        #print(actVotesOfCandidatesDictI)

        # select candidate with highest number of votes
        #selectedCandidateI:int = AggrDHont.selectorOfTheMostVotedItem(actVotesOfCandidatesDictI)
        v = np.argmax(list(actVotesOfCandidatesDictI.values()))
        print(actVotesOfCandidatesDictI)
        selectedCandidateI:int = list(actVotesOfCandidatesDictI.keys())[v]
        print("SelectedCandidate:",selectedCandidateI)
        
        # add new selected candidate in results
        recommendedItemIDs.append(selectedCandidateI);

        # removing elected candidate from list of candidates
        uniqueCandidatesI.remove(selectedCandidateI)

        # updating number of elected candidates of parties
        electedOfPartyDictI:dict = {partyIDI:electedOfPartyDictI[partyIDI] + methodsResultDict[partyIDI].get(selectedCandidateI, 0) for partyIDI in electedOfPartyDictI.keys()}
        totalSelectedCandidatesVotes = np.sum(list(electedOfPartyDictI.values()))
        print("electedOfPartyDictI: ", electedOfPartyDictI, totalSelectedCandidatesVotes)
        
        
      # list<int>
      return recommendedItemIDs[:numberOfItems]


In [66]:
run(methodsResultDict, modelDF, userID, 5)

UniqueCandidatesI:  [1, 2, 3, 4, 5, 6]
ElectedForPartyI:  {'method1': 0.0, 'method2': 0.0, 'method3': 0.0}
VotesOfPartiesDictI:  {'method1': 0.5, 'method2': 0.4, 'method3': 0.1} 1.0
{1: 0.9, 2: 0.25, 3: 0.51, 4: 0.25, 5: 0.36000000000000004, 6: 0.2}
SelectedCandidate: 1
electedOfPartyDictI:  {'method1': 0.5, 'method2': 0.5, 'method3': 0.0} 1.0
{2: 0.25, 3: 0.6100000000000001, 4: 0.35000000000000003, 5: 0.26, 6: 0.30000000000000004}
SelectedCandidate: 3
electedOfPartyDictI:  {'method1': 0.9, 'method2': 0.5, 'method3': 0.7} 2.0999999999999996
{2: 0.3999999999999998, 4: 0.2, 5: 0.7, 6: 0.1}
SelectedCandidate: 5
electedOfPartyDictI:  {'method1': 0.9, 'method2': 1.4, 'method3': 0.7} 3.0
{2: 0.5, 4: 0.2, 6: 0.1}
SelectedCandidate: 2
electedOfPartyDictI:  {'method1': 1.4, 'method2': 1.4, 'method3': 0.7} 3.5
{4: 0.2, 6: 0.1}
SelectedCandidate: 4
electedOfPartyDictI:  {'method1': 1.5999999999999999, 'method2': 1.4, 'method3': 1.0} 4.0


[1, 3, 5, 2, 4]

In [52]:
methodsResultDict

{'method1': {1: 0.5, 2: 0.5, 3: 0.4, 4: 0.2},
 'method2': {5: 0.9, 1: 0.5, 6: 0.1},
 'method3': {3: 0.7, 4: 0.3, 6: 0.9}}